# Camera Calibration

In [ ]:
import os
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
from lib import app
from lib.extract import get_frames
from lib.utils import load_points, save_points

plt.style.use(os.path.join('..', 'configs', 'mplstyle.yaml'))

%load_ext autoreload
%autoreload 2

DATA_ROOT_DIR = os.path.join('..', 'data')

# Intrinsic Calibration
## 0. Define Params

In [ ]:
YEAR = 2019

# checkerboard info
INTRINSIC_BOARD_SHAPE = (9, 6) # (horizontal, vertical) num of inner corners of checkerboard pattern
INTRINSIC_SQUARE_LEN = 0.04    # length of one side of black/white chessboard square in metres

# automatically defined params - do not modify
INTRINSIC_DATA_DIR = os.path.join(DATA_ROOT_DIR, 'intrinsic_calib', str(YEAR))

## 1. Extract frames
You must first define the video you wish to use for calibration. Eg. for 2019, you can define video_fname to be '1.mp4' or '2.mp4'.
GUI usage:
- `.` - Next frame
- `,` - Previous frame
- `s` - Save current frame
- `q` - Quit

In [ ]:
video_fname = '1.mp4'

intrinsic_video_path = os.path.join(INTRINSIC_DATA_DIR, 'videos', video_fname)
frames_fpath = os.path.join(INTRINSIC_DATA_DIR, 'frames')
if not os.path.exists(frames_fpath):
    os.makedirs(frames_fpath)
    
get_frames(
    vid_fpath=intrinsic_video_path, 
    frame_output_dir=frames_fpath
)

## 2. Find calibration board corners
You must first define OpenCV's window size. You may need to rerun this cell with various window sizes until you are happy with the accuracy of the results

In [ ]:
window_size = 5 # pixels

# prepare the output dir
frames_fpath = os.path.join(INTRINSIC_DATA_DIR, 'frames')
if not os.path.exists(frames_fpath):
    os.makedirs(frames_fpath)
    
# 'window_size' sets the size of the calibration board corner detector window size
app.extract_corners_from_images(
    img_dir=frames_fpath, 
    out_fpath=os.path.join(INTRINSIC_DATA_DIR, 'points.json'), 
    board_shape=INTRINSIC_BOARD_SHAPE, 
    board_edge_len=INTRINSIC_SQUARE_LEN, 
    window_size=window_size, 
    remove_unused_images=False
)

## 3. Plot detected points

In [ ]:
app.plot_corners(os.path.join(INTRINSIC_DATA_DIR, 'points.json'))

# 4. Calibrate
If you are not satisfied with the calibration error, you can remove frames with inaccurate checkerboard points. A feature that shows you how much error each frame adds to the calibration is in the pipeline, but until then you have to determine those frames manually and then remove them

In [ ]:
K, D, R, t, used_points, rms = app.calibrate_fisheye_intrinsics(
    points_fpath=os.path.join(INTRINSIC_DATA_DIR, 'points.json'), 
    out_fpath=os.path.join(INTRINSIC_DATA_DIR, 'camera.json')
)
print(f'\nRMS Error is {rms:.3f} pixels')

## 5. Plot undistorted points using newly-found intrinsic parameters

In [ ]:
scene = app.plot_points_fisheye_undistort(
    points_fpath=os.path.join(INTRINSIC_DATA_DIR, 'points.json'), 
    camera_fpath=os.path.join(INTRINSIC_DATA_DIR, 'camera.json')
)

# Extrinsic calibration
## 0. Define params

In [ ]:
# the path to the scene you wish to calibrate
DATA_DIR = os.path.join(DATA_ROOT_DIR, '2019_02_27')
# DATA_DIR = os.path.join(DATA_ROOT_DIR, '2017_08_29', 'top')

# Extrinsic checkerboard info
EXTRINSIC_BOARD_SHAPE = (9, 6)
EXTRINSIC_SQUARE_LEN = 0.088 # meters

### Automatically defined params
Do not modify

In [ ]:
EXTRINSIC_DATA_DIR = os.path.join(DATA_DIR, 'extrinsic_calib')
DUMMY_SCENE = os.path.abspath(os.path.join('..', 'configs', 'dummy_scene.json'))

# Camera settings
VID_FPATHS = sorted(glob(os.path.join(EXTRINSIC_DATA_DIR, 'videos', 'cam[1-9].mp4'))) # paths to the calibration videos

CAMS = [int(os.path.splitext(os.path.basename(cam))[0][-1]) for cam in VID_FPATHS] # must be sorted
print('Cams:', CAMS)

# Intrinsic calibration
YEAR = 2017 if '2017' in DATA_DIR else 2019
INTRINSIC_DATA_DIR = os.path.join(DATA_ROOT_DIR, 'intrinsic_calib', str(YEAR))

## 1. Extract frames from videos
You must first define the camera from which you wish to extract frames. ```camera``` must correspond to one of the numbers in ```CAMS```. Thereafter, use the GUI to save only those frames where the checkerboard squares are visible.

GUI usage:
- `.` - Next frame
- `,` - Previous frame
- `s` - Save current frame
- `q` - Quit

In [ ]:
camera = 1  # Change as needed (in this example we have cameras 1,2,...
assert camera in CAMS

frames_fpath = os.path.join(EXTRINSIC_DATA_DIR, 'frames', str(camera))
if not os.path.exists(frames_fpath):
    os.makedirs(frames_fpath)
print('The the output folder is', frames_fpath)

get_frames(VID_FPATHS[camera-1], frames_fpath)
# cv.waitKey(0); # Needed to close window properly
print('GREAT JOB!!')

### Optional: Convert frames to negatives
This is needed for the days that use the checkerboard with a black outline so that OpenCV can detect the checkerboard points correctly

In [ ]:
for cam in CAMS:
    frames_fpath = os.path.join(EXTRINSIC_DATA_DIR, 'frames', str(cam))
    neg_frames_dir = os.path.join(EXTRINSIC_DATA_DIR, 'neg_frames', str(cam))
    if not os.path.exists(neg_frames_dir):
        os.makedirs(neg_frames_dir)
    print('The the output folder is', neg_frames_dir)
    
    for fname in os.listdir(frames_fpath):
        fname = fname.lower()
        if fname.endswith('.jpg') or fname.endswith('.png'):
            img = cv.imread(os.path.join(frames_fpath, fname))
            img_neg = (255 - img)
            cv.imwrite(os.path.join(neg_frames_dir, fname), img_neg)

## 2. Find calibration board corners
Note: This takes a while!

You must first define OpenCV's window size. You may need to rerun this cell with various window sizes until you are happy with the accuracy of the results.
If the checkerboard with the black outline is used in the calibration vids, change ```'frames'``` to ```'neg_frames'```

In [ ]:
window_size = 5

# set directories
points_dir = os.path.join(EXTRINSIC_DATA_DIR, 'points')
if not os.path.exists(points_dir):
    os.makedirs(points_dir)
data_dirs = [[
    os.path.join(EXTRINSIC_DATA_DIR, 'frames', str(cam)),
    os.path.join(points_dir, f'points{cam}.json')
] for cam in CAMS]

# Find calibration board corners in images and save points
for [img_dir, out_fpath] in data_dirs:
    # 'window_size' sets the size of the calibration board corner detector window size
    app.extract_corners_from_images(
        img_dir, 
        out_fpath,
        EXTRINSIC_BOARD_SHAPE,
        EXTRINSIC_SQUARE_LEN, 
        window_size=window_size, 
        remove_unused_images=False
    )

### Correct points that were detected in the reversed order
Sometimes OpenCV detects a frame's checkerboard points in the reverse direction relative to other frames. Use this code to correct those points.

For each cam, you must manually insert the frame number that has reversed points. Eg. if img00012.jpg and img00100.jpg from cam3 has reversed points then ```frames = [[],
[],
[12, 100],
[],
[],
[]]```
(assuming there are 6 cams in this example)

In [ ]:
points_fpaths = sorted(glob(os.path.join(EXTRINSIC_DATA_DIR, 'points', 'points[1-9].json')))
print(points_fpaths)

# list of frames where checkerboard points were detected in the wrong orientation
frames = [[],
         [],
         [],
         [],
         [],
         []]
assert len(frames) == len(points_fpaths)

for i in range(len(frames)):
    if frames[i]:
        points, fnames, board_shape, board_edge_len, cam_res = load_points(points_fpaths[i])
        for f in frames[i]:
            img_name = f'img{f:05}.jpg'
            index = fnames.index(img_name)
            img_pts = points[index]
            points[index] = np.flip(img_pts, (0, 1))
        save_points(points_fpaths[i], points, fnames, board_shape, board_edge_len, cam_res)

## 3. Calibrate pairwise extrinsics
If one or more cam pairs do not have common image points, you need to use Argus Clicker to define manual points. Thereafter you must rerun the cell below to finalise the extrinsics. If the manual points minimization did not yield satisfactory extrinsic results, you have to adjust the ```redescending_loss``` params in ```calib.adjust_extrinsics_manual_points``` by trial and error (for now) until the results are satisfactory

In [ ]:
camera_fpaths = [os.path.join(INTRINSIC_DATA_DIR, 'camera.json')]*len(CAMS)
points_fpaths = sorted(glob(os.path.join(EXTRINSIC_DATA_DIR, 'points','points[1-9].json')))
scene_fpath = os.path.join(EXTRINSIC_DATA_DIR, f'{len(points_fpaths)}_cam_scene.json')

app.calibrate_fisheye_extrinsics_pairwise(
    camera_fpaths, points_fpaths, 
    out_fpath=scene_fpath,# cams=CAMS,
    dummy_scene_fpath=DUMMY_SCENE
)

## 4. Run calibration SBA
Note: Also takes a while!

In [ ]:
scene_sba_fpath = scene_fpath.replace('.json','_sba.json')

res = app.sba_board_points_fisheye(
    scene_fpath, points_fpaths, out_fpath=scene_sba_fpath, 
    manual_points_fpath=manual_points_fpath#, manual_points_only=True,
    # camera_indices=None
)

plt.plot(res['before'], label='Cost before')
plt.plot(res['after'], label='Cost after')
plt.legend()
plt.show()

## 5. Visualize
### Plot checkerboard points

In [ ]:
app.plot_scene(DATA_DIR, dark_mode=True)
# Optionally, specify a certain scene file to view (points may be reconstructed incorrectly)
# app.plot_scene(DATA_DIR, scene_fname='6_cam_scene_before_corrections.json', dark_mode=True)

### Plot manually defined points

In [ ]:
app.plot_scene(DATA_DIR, manual_points_only=True, dark_mode=True)